In [8]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

  Using cached langchain_core-0.1.53-py3-none-any.whl.metadata (5.9 kB)
Using cached langchain_core-0.1.53-py3-none-any.whl (303 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.34
    Uninstalling langchain-core-0.3.34:
      Successfully uninstalled langchain-core-0.3.34
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.0.9 requires langchain-core<0.4.0,>=0.3.27, but you have langchain-core 0.1.53 which is incompatible.


In [13]:
!pip install langchain-core==0.1.53

  Using cached langchain_core-0.1.53-py3-none-any.whl.metadata (5.9 kB)
Using cached langchain_core-0.1.53-py3-none-any.whl (303 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.34
    Uninstalling langchain-core-0.3.34:
      Successfully uninstalled langchain-core-0.3.34
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.0.9 requires langchain-core<0.4.0,>=0.3.27, but you have langchain-core 0.1.53 which is incompatible.


In [14]:
!pip install langchain-google-genai==1.0.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.1/679.1 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 26.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.3
    Uninstalling protobuf-5.29.3:
      Successfully uninstalled protobuf-5.29.3
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.10
    Uninstalling google-ai-generativelanguage-0.6.10:
      Successfully uninstalled google-ai-generativelanguage-0.6.10
  Attempting uninstall: google-generativeai
    Found existing installation: google-generativeai 0.8.3
    Uninstalling google-generativeai-0.8.3:
      Successfully uninstalled google-generativeai-0.8.3
  Attempting uninstall: langchain-google-genai
    Found existing installation: langchain-google-genai 2.0.9
    Uninstalling langchain-google-genai

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [4]:
def get_openai_api_key() -> str:
  return ''

def get_serper_api_key() -> str:
  return ''

def get_google_api_key() -> str:
  return ''

In [5]:
import os

# openai_api_key = get_openai_api_key()
# os.environ["OPENAI_API_KEY"] = get_openai_api_key()
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()
os.environ["GOOGLE_API_KEY"] = get_google_api_key()

In [6]:
# call gemini model
llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash',
                            verbose=True,
                            temperature=0.5,
                            goggle_api_key=get_google_api_key())

Create Agents

In [30]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at Blue Prism, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm,
)

In [31]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of Blue Prism's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm
)

In [32]:
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool

In [33]:
directory_read_tool = DirectoryReadTool(
    directory='./instructions',
    config=dict(
        llm=dict(
            provider="google",
            config=dict(
                model="gemini-flash-1.5",
                temperature=0.5,
            ),
        ),
        embedder=dict(
          provider="google",
          config=dict(
              model="models/embedding-001",
              task_type="retrieval_document",
          )
        ),
      ),
    )

file_read_tool = FileReadTool(
        config=dict(
        llm=dict(
            provider="google",
            config=dict(
                model="gemini-flash-1.5",
                temperature=0.5,
            ),
        ),
        embedder=dict(
          provider="google",
          config=dict(
              model="models/embedding-001",
              task_type="retrieval_document",
          )
        ),
      ),
)

search_tool = SerperDevTool(
        config=dict(
        llm=dict(
            provider="google",
            config=dict(
                model="gemini-flas-1.5",
                temperature=0.5,
            ),
        ),
        embedder=dict(
          provider="google",
          config=dict(
              model="models/embedding-001",
              task_type="retrieval_document",
          )
        ),
      ),
)

custom tool

In [34]:
from crewai_tools import BaseTool

In [35]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")

    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [36]:
sentiment_analysis_tool = SentimentAnalysisTool()

Tasks

In [37]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    # tools=[],
    agent=sales_rep_agent,
)

In [38]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool],
    agent=lead_sales_rep_agent,
)

Create Crew

In [39]:
crew = Crew(
    agents=[sales_rep_agent, lead_sales_rep_agent],
    tasks=[lead_profiling_task, personalized_outreach_task],
    verbose=2,
    memory=False
)

Running the Crew

In [40]:
inputs = {
    "lead_name": "Saraca Laborataries Limited",
    "industry": "Drug Manufacturing",
    "key_decision_maker": "Sunil Lingareddy",
    "position": "CEO",
    "milestone": "regulatory compliance"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of Saraca Laborataries Limited, a company in the Drug Manufacturing sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
Thought: I need to find information about Saraca Laboratories Limited.  I don't have any files to start with, so I'll search the internet for information.

Action: Search the internet

Action Input: {"search_query": "Saraca Laboratories Limited"}
 


Search results: Title: Saraca Laboratories Limited – Manufacturerers of Basic Drugs ...
Link: http://saraca.com/
Snippet: Saraca Laboratories Limited 

In [41]:
from IPython.display import Markdown
Markdown(result)

**Email 1: Subject: Streamlining Operations & Regulatory Compliance at Saraca Laboratories Limited**

Dear Mr. Sunil Lingareddy,

My name is [Your Name], Lead Sales Representative at Blue Prism. I'm reaching out to Saraca Laboratories Limited because of your company's impressive presence in the pharmaceutical industry, manufacturing and distributing Active Pharmaceutical Ingredients (APIs) and Intermediates across 40 countries.  We understand the significant regulatory demands and operational complexities within this sector.

Blue Prism's intelligent automation solutions are designed to address these challenges head-on.  Our technology helps pharmaceutical companies like yours improve efficiency, reduce errors, and enhance regulatory compliance across various processes, from manufacturing and supply chain management to data management and reporting.

We've helped numerous pharmaceutical companies achieve significant improvements in operational efficiency and regulatory adherence.  We would be delighted to share specific case studies and discuss how Blue Prism can help Saraca Laboratories Limited achieve similar results.

Would you be open to a brief introductory call next week to explore this further?

Sincerely,

[Your Name]
Lead Sales Representative
Blue Prism


**Email 2: Subject: Enhancing Efficiency and Compliance at Saraca Laboratories Limited - Following Up**

Dear Mr. Sunil Lingareddy,

Following up on my previous email, I wanted to share some additional information on how Blue Prism can specifically benefit Saraca Laboratories Limited.

Given your global reach and the complexity of the pharmaceutical industry, intelligent automation can significantly improve your:

* **Regulatory Compliance:**  Automate tasks related to documentation, reporting, and data management to ensure seamless adherence to stringent regulations.
* **Manufacturing Efficiency:** Streamline processes such as inventory management, quality control, and order fulfillment, reducing errors and improving overall productivity.
* **Supply Chain Optimization:** Enhance visibility and control across your supply chain, improving efficiency and reducing costs.

I've attached a brief case study showcasing how a similar pharmaceutical company improved its operational efficiency by X% and reduced compliance-related costs by Y% using Blue Prism.

I'm available for a call at your earliest convenience to discuss your specific needs and how Blue Prism can help Saraca Laboratories Limited achieve its goals.

Sincerely,

[Your Name]
Lead Sales Representative
Blue Prism


**Email 3: Subject:  Meeting Proposal: Optimizing Saraca Laboratories Limited's Operations with Blue Prism**

Dear Mr. Sunil Lingareddy,

I hope this email finds you well.

Following up on our previous conversations, I'd like to propose a brief meeting to discuss how Blue Prism can help Saraca Laboratories Limited achieve its operational and regulatory compliance goals.  We can tailor a demonstration to highlight the specific benefits relevant to your business, focusing on areas such as [mention 1-2 specific areas based on further research if possible, otherwise leave blank].

Please let me know what time and day works best for you.  I'm available on [list available dates/times].

Thank you for your time and consideration.

Sincerely,

[Your Name]
Lead Sales Representative
Blue Prism